<a href="https://colab.research.google.com/github/valeriocardoso/learning-area/blob/master/Machine_Learning/DGL-Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl

     |████████████████████████████████| 3.0MB 4.3MB/s 


In [2]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [5]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [6]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [7]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Extracting file to /root/.dgl/cora


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 2.0084 | Test Acc 0.1160 | Time(s) nan
Epoch 00001 | Loss 1.8265 | Test Acc 0.3430 | Time(s) nan
Epoch 00002 | Loss 1.6620 | Test Acc 0.3540 | Time(s) nan
Epoch 00003 | Loss 1.5301 | Test Acc 0.3690 | Time(s) 0.1448
Epoch 00004 | Loss 1.4189 | Test Acc 0.4070 | Time(s) 0.1465
Epoch 00005 | Loss 1.3242 | Test Acc 0.4510 | Time(s) 0.1458
Epoch 00006 | Loss 1.2394 | Test Acc 0.4860 | Time(s) 0.1455
Epoch 00007 | Loss 1.1629 | Test Acc 0.5260 | Time(s) 0.1457
Epoch 00008 | Loss 1.0936 | Test Acc 0.5590 | Time(s) 0.1454
Epoch 00009 | Loss 1.0302 | Test Acc 0.5830 | Time(s) 0.1452
Epoch 00010 | Loss 0.9721 | Test Acc 0.6230 | Time(s) 0.1450
Epoch 00011 | Loss 0.9185 | Test Acc 0.6480 | Time(s) 0.1454
Epoch 00012 | Loss 0.8695 | Test Acc 0.6600 | Time(s) 0.1454
Epoch 00013 | Loss 0.8242 | Test Acc 0.6700 | Time(s) 0.1453
Epoch 00014 | Loss 0.7814 | Test Acc 0.6770 | Time(s) 0.1451
Epoch 00015 | Loss 0.7404 | Test Acc 0.6890 | Time(s) 0.1452
Epoch 00016 | Loss 0.7009 | Test 